In [5]:
import cv2
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import compress

import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian,unary_from_softmax

import subprocess

from glob import glob
import os
from tqdm import tqdm
from numpy.lib.stride_tricks import as_strided
import csv

In [ ]:
IMAGES_FOLDER_PATH = "../datasets/custom_still"


# Point Clouds 
INITIAL_POINT_CLOUD = '../output/initial_point_cloud.ply'
FINAL_POINT_CLOUD = '../output/final_point_cloud.ply'

# Bundle File
BUNDLE_FILE = '../output/bundle.out'

# Shi-Tomasi parameters
feature_params = dict(maxCorners = 5000, 
                      qualityLevel = 0.03, 
                      minDistance = 10, 
                      blockSize = 15
                      )

# Lucas-Kanade parameters
lk_params = dict(   winSize  = (25,25),
                    maxLevel = 8,
                    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.3))
# Ceres-Solver parameters
CERES_PARAMS = dict(
                    solver = '../ceres-bin/bin/bundle_adjuster',
                    maxIterations = 1000,
                    input_ply = '../output/initial.ply',
                    output_ply = '../output/final.ply',
                    inner_iterations = 'true',
                    nonmonotonic_steps = 'false'
                    )

CAMERA_PARAMS = dict(fx=1781,
                     fy=1781,
                     cx=960,
                     cy=540,
                     k1=0,
                     k2=0,
                     s=0,
                    )


In [6]:
def Sad(ref_patch, warp_patch) :

    '''
    Calculates L1 Loss between two grayscale image patches
    N : Total number of warp images
    P : Total number of patches per image
    w : Length of one side of patch

    Dimension of patch ndarray : N x P x (w*w)
    Returned array dim : N x P
    '''

    err = np.sum(np.abs(warp_patch - ref_patch), axis=2)
    return err


def HomographyFrom(K, C1, R1, C2, R2, dep):

    # C1, R1 : Reference Image
    H  = dep * K @ R2 @ R1.T @ np.linalg.inv(K)
    H[:,2] += K @ R2 @ (C1 - C2)
    return H


def MergeScores(scores, valid_ratio = 0.5):
    '''
    Takes the average of top k values in array. k == valid_scores.
    N : Total number of warp images
    P : Total number of patches per image

    Dimension of scores array: N x P
    Dimension of returned array: (N*valid_ratio) x P
    '''

    num_valid_scores = int(scores.shape[0] * valid_ratio)

    ix = np.argpartition(scores, num_valid_scores, axis=0)
    ix = ix[:num_valid_scores,:]

    srt = np.take_along_axis(scores, ix, axis=0)
    score = np.sum(srt, axis=0) / num_valid_scores

    return score

def GetMin(values, size):
    '''
    Get smallest two values in array
    '''

    assert(size>1)

    f = 0
    s = 0

    f, s = np.partition(values, 1)[0:2]

    return f, s


def Modulate(cost_volume_arr):

    first = 0
    second = 0
    confidence = 0
    num_samples = cost_volume_arr.shape[0]

    for r in range(cost_volume_arr.shape[1]):
        for c in range(cost_volume_arr.shape[2]):

            values = cost_volume_arr[:, r, c]
            first, second = GetMin(values, num_samples)
            confidence = (second + 1) / (first + 1)
            cost_volume_arr[:, r, c] = values * confidence

    return cost_volume_arr

def plane_sweep(folder, outfile, depth_samples, min_depth, max_depth, scale, patch_radius):

    print(f"Number of depth samples: {depth_samples.shape[0]}")

    # Intrinsics, Camera centers, Rotation mtx
    K = utilities.construct_camera_matrix(config.CAMERA_PARAMS)
    C = []
    R = []

    # Get extrinsics
    with open(config.EXTRINSIC_FILE) as ext_file:
        csv_reader = csv.reader(ext_file, delimiter=',')

        for row in csv_reader:

            p = [float(r) for r in row[:-1]]
            rot, _ = cv2.Rodrigues(np.array(p[:3]))
            trans = np.array(p[3:6])
            c = -1 * np.linalg.inv(rot) @ trans

            C.append(c)
            R.append(rot)

    # Get all images
    # total_images = config.NUM_IMAGES
    all_img = []
    for file in sorted(os.listdir(config.IMAGE_DIR))[:len(R)] :  # Get as many images as the extrinsics available

        if file.endswith('.png') or file.endswith('.jpg') :
            im = cv2.imread(os.path.join(config.IMAGE_DIR, file))
            all_img.append(im)

    scaled_gray_images = []
    for img in all_img :
        img = img.astype(np.float32)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        for s in range(scale):
            gray_img = cv2.pyrDown(gray_img)

        scaled_gray_images.append(gray_img)

    ref_img = scaled_gray_images[0]
    height, width = ref_img.shape

    num_images = len(all_img)
    cost_volume_arr = np.zeros((depth_samples.shape[0], height, width))

    for idx, depth in enumerate(tqdm(depth_samples)):

        homographies = np.zeros((num_images, 3, 3))
        warped_images = []

        for ind in range(num_images) :

            h = HomographyFrom(K, C[0], R[0], C[ind], R[ind], depth)
            actual_scale = 2**scale
            h[:,:2] *= actual_scale
            h[2,:] *= actual_scale
            homographies[ind,:,:] = h

        # Assume 0th image is reference image
        for i in range(1, num_images):
            warp = cv2.warpPerspective(scaled_gray_images[i], homographies[i], ref_img.shape[::-1], cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
            warped_images.append(warp)


        ref_img_patches = as_strided(ref_img, shape=(ref_img.shape[0] - 2*patch_radius,
                                    ref_img.shape[1] - 2*patch_radius, 2*patch_radius + 1, 2*patch_radius + 1),
                                    strides=ref_img.strides + ref_img.strides, writeable=False)

        h, w, _, _ = ref_img_patches.shape
        patch_size = 2*patch_radius + 1
        ref_img_patches = ref_img_patches.reshape((ref_img_patches.shape[0]*ref_img_patches.shape[1], patch_size**2))
        warp_patches = np.zeros((len(warped_images), ref_img_patches.shape[0], ref_img_patches.shape[1]))
        for i in range(len(warped_images)):

            x = as_strided(warped_images[i], shape=(warped_images[i].shape[0] - 2*patch_radius,
                            warped_images[i].shape[1] - 2*patch_radius, 2*patch_radius + 1, 2*patch_radius + 1),
                            strides=warped_images[i].strides + warped_images[i].strides, writeable=False)

            x = x.reshape((x.shape[0]*x.shape[1], patch_size**2))
            warp_patches[i,:,:] = x

        L1_diff = Sad(ref_img_patches, warp_patches)
        score = MergeScores(L1_diff, valid_ratio = 0.5)

        # Border pixels take values of the neighboring pixels
        cost_volume_arr[idx, patch_radius:height-patch_radius, patch_radius:width-patch_radius] = score.reshape((h,w))
        cost_volume_arr[idx, 0: patch_radius, :] = cost_volume_arr[idx, patch_radius, :]
        cost_volume_arr[idx, height-patch_radius+1:, :] = cost_volume_arr[idx, height-patch_radius, :]
        cost_volume_arr[idx, :, 0: patch_radius] = cost_volume_arr[idx, :, patch_radius].reshape((cost_volume_arr[idx, :, patch_radius].shape[0],1))
        cost_volume_arr[idx, :, width-patch_radius+1:] = cost_volume_arr[idx, :, width-patch_radius].reshape((cost_volume_arr[idx, :, width-patch_radius].shape[0],1))

    cost_volume_arr = Modulate(cost_volume_arr)

    # Saving convention
    np.savez_compressed(outfile, pc_cost=cost_volume_arr, dir=folder, max_d=max_depth, min_d=min_depth)

    return cost_volume_arr.astype('float32')


In [7]:
def compute_unary_image(unary, depth_samples, outfile):

	gd = np.argmin(unary, axis=0)
	gd_im = np.zeros((unary.shape[1], unary.shape[2]))
	for i in range(unary.shape[1]):
		for j in range(unary.shape[2]):
			gd_im[i,j] = ((depth_samples[gd[i,j]] - np.min(depth_samples)) * 255.0) / (np.max(depth_samples) - np.min(depth_samples))

	cv2.imwrite(outfile, gd_im)

def DenseCRF(unary, img, depth_samples, params, folder, max_depth, min_depth, outfile='depth_map.png', show_wta=False):

	labels = unary.shape[0]
	iters = params['iters']
	weight = params['weight']
	pos_std = params['pos_std']
	rgb_std = params['rgb_std']
	max_penalty = params['max_penalty']

	# Get initial crude depth map from photoconsistency
	if show_wta :
		compute_unary_image(unary, depth_samples, outfile=f'../output/{folder}/cost_volume_{depth_samples.shape[0]}_wta.png')

	# Normalize values for each pixel location
	for r in range(unary.shape[1]):
		for c in range(unary.shape[2]):
			if np.sum(unary[:, r, c]) <= 1e-9:
				unary[:, r, c] = 0.0
			else:
				unary[:, r, c] = unary[:, r, c]/np.sum(unary[:, r, c])

	# Convert to class probabilities for each pixel location
	unary = unary_from_softmax(unary)

	d = dcrf.DenseCRF2D(img.shape[1], img.shape[0], labels)

	# Add photoconsistency score as uanry potential. 16-size vector
	# for each pixel location
	d.setUnaryEnergy(unary)
	# Add color-dependent term, i.e. features are (x,y,r,g,b)
	d.addPairwiseBilateral(sxy=pos_std, srgb=rgb_std, rgbim=img, compat=np.array([weight, labels*max_penalty]), kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

	# Run inference steps
	Q = d.inference(iters)

	# Extract depth values. Map to [0-255]
	MAP = np.argmax(Q, axis=0).reshape((img.shape[:2]))
	depth_map = np.zeros((MAP.shape[0], MAP.shape[1]))

	for i in range(MAP.shape[0]):
		for j in range(MAP.shape[1]):
			depth_map[i,j] = depth_samples[MAP[i,j]]

	min_val = np.min(depth_map)
	max_val = np.max(depth_map)

	for i in range(MAP.shape[0]):
		for j in range(MAP.shape[1]):
			depth_map[i,j] = ((depth_map[i,j] - min_val)/(max_val - min_val)) * 255.0

	# Upsampling depth map
	# depth_map = cv2.resize(depth_map, (config.CAMERA_PARAMS['cx'] * 2,config.CAMERA_PARAMS['cy'] * 2), interpolation=cv2.INTER_LINEAR)
	cv2.imwrite(outfile, depth_map)

In [13]:
def dense_depth(args) :

    folder = args['folder']
    num_samples = int(args['nsamples'])
    # 	pc_path = args['pc']
    show_wta = args['show_wta']

    scale = int(args['scale'])
    max_depth = float(args['max_d'])
    min_depth = float(args['min_d'])
    patch_radius = int(args['patch_rad'])

    pc_score = 0

    # Create depth samples in the specified depth range
    depth_samples = np.zeros(num_samples)
    step = step = 1.0 / (num_samples - 1.0)

    for val in range(num_samples):
        sample = (max_depth * min_depth) / (max_depth - (max_depth - min_depth) * val * step)
        depth_samples[val] = config.CAMERA_PARAMS['fx']/sample
        # depth_samples[val] = sample

    # Get reference image
    file = ''
    for f in sorted(os.listdir(config.IMAGE_DIR)):
        if f.endswith('.png') or f.endswith('.jpg'):
            file = f
            break

    ref_img = cv2.imread(os.path.join(config.IMAGE_DIR.format(folder), file))

    for s in range(scale):
        ref_img = cv2.pyrDown(ref_img)
    # Mean shifting image
    ref_img = cv2.pyrMeanShiftFiltering(ref_img, 20, 20, 1)

    ref_img = cv2.cvtColor(ref_img, cv2.COLOR_BGR2Lab)

    # Perform plane sweep to calculate photo-consistency loss
    outfile = f'../output/cost_volume_{depth_samples.shape[0]}'
    print("Doing Plane Sweep Calculation for photoconsistency")
    pc_score = plane_sweep(folder, outfile, depth_samples, min_depth, max_depth, scale, patch_radius)
    print("Finished Plane Sweep Calculation")

    outfile = f'../output/cost_volume_{depth_samples.shape[0]}__{config.CRF_PARAMS["rgb_std"]}_depth_map.png'
    crf_params = dict()
    crf_params['iters'] = int(args['iters'])
    crf_params['pos_std'] = tuple(float(x) for x in args['p_std'].split(','))
    crf_params['rgb_std'] = tuple(float(x) for x in args['c_std'].split(','))
    crf_params['weight'] = float(args['wt'])
    crf_params['max_penalty'] = float(args['max_p'])

    # Use photoconsistency score as unary potential
    print("Dense Map calculation")
    depth_map = DenseCRF(pc_score, ref_img, depth_samples, crf_params, folder, max_depth, min_depth, outfile, show_wta)
    print("Finished solving")

In [ ]:
args = dict()
